In [1]:
import math
import numpy as np

In [2]:
def readTxtFile(inputFile):
    myFile = open(inputFile, "r")
    lines = []
    variables = dict()
    for line in myFile:
        line = line.strip()
        temp = line.split(" ")
        lines.append(temp)

    delx = float(lines[0][0])
    dely = float(lines[0][1])
    dx = float(lines[0][2])
    dy = float(lines[0][3])
    sigmaSquared = float(lines[0][4])
    M = int(lines[1][0])
    N = int(lines[1][1])
    bigImage = []
    for row in lines[2:12]:
        bigImage.append([])
        for pixel in row:
            bigImage[-1].append(float(pixel))
    m = int(lines[12][0])
    n = int(lines[12][1])
    smallImage = []
    for row in lines[13:19]:
        smallImage.append([])
        for pixel in row:
            smallImage[-1].append(float(pixel))
    return(delx,dely,dx,dy,sigmaSquared,M,N,bigImage,m,n,smallImage)

In [3]:
# Intensity of the image at a real-valued point (x', y') by the interpolation formula 
# I(A, x', y')
def interpolationFormula(M, N, sigmaSquared, A, x, y):
    numerator = 0
    denominator =0 
    for p in range(M):
        for q in range(N):
            difference = (x-p)**2 + (y-q)**2
            numerator += A[p][q]*math.exp(-sigmaSquared*difference)
            denominator += math.exp(-sigmaSquared*difference)
    if denominator > 0:
        intensity = numerator/denominator
    else:
        intensity = 0
    
    return intensity 

In [4]:
# f(A, T, x, y)
def objectiveFunction(M, N, sigmaSquared, m, n, A, T, x, y):
    optimizationSum = 0
    for i in range(m):
        for j in range(n):
            interpolatedIntensity = interpolationFormula(M, N, sigmaSquared, A, i+x, j+y)
            optimizationSum += (T[i][j] - interpolatedIntensity)**2
    return optimizationSum


In [5]:
# 4a
def gradient(M, N, sigmaSquared, dx, dy, m, n, A, T, x, y):
    secondOrderXSum = 0
    secondOrderYSum = 0
    for i in range(m):
        for j in range(n):
            firstTermX = (T[i][j]-interpolationFormula(M, N, sigmaSquared, A, i+x+dx, j+y))**2
            secondTermX = (T[i][j]-interpolationFormula(M, N, sigmaSquared, A, i+x-dx, j+y))**2
            secondOrderXSum += firstTermX-secondTermX
            firstTermY = (T[i][j]-interpolationFormula(M, N, sigmaSquared, A, i+x, j+y+dy))**2
            secondTermY = (T[i][j]-interpolationFormula(M, N, sigmaSquared, A, i+x, j+y-dy))**2
            secondOrderYSum += firstTermY-secondTermY
    secondOrderX = secondOrderXSum/(2*dx)
    secondOrderY = secondOrderYSum/(2*dy)

    return(secondOrderX, secondOrderY)

In [6]:
# 4b
def calculateHessian(M, N, sigmaSquared, dx, dy, m, n, A, T, x, y):
    # doble derivative wrt x 
    num = (objectiveFunction(M, N, sigmaSquared, m, n, A, T, x+dx, y) + 
            objectiveFunction(M, N, sigmaSquared, m, n, A, T, x-dx, y) - 
            2*objectiveFunction(M, N, sigmaSquared, m, n, A, T, x, y)
    )
    den = dx**2
    doubDerWRTx = num/den

    # double derivative wrt y 
    num2 = (objectiveFunction(M, N, sigmaSquared, m, n, A, T, x, y+dy) + 
        objectiveFunction(M, N, sigmaSquared, m, n, A, T, x, y-dy) -
        2*objectiveFunction(M, N, sigmaSquared, m, n, A, T, x, y)
    )
    den2 = dy**2 
    doubDerWRTy = num2/den2

    # double derivative wrt to both x and y 
    numerator = (objectiveFunction(M, N, sigmaSquared, m, n, A, T, x+dx, y+dy) - 
                objectiveFunction(M, N, sigmaSquared, m, n, A, T, x+dx, y-dy) -
                objectiveFunction(M, N, sigmaSquared, m, n, A, T, x-dx, y+dy) + 
                objectiveFunction(M, N, sigmaSquared, m, n, A, T, x-dx, y-dy)
    )
    denominator = 4*dy*dx
    doubleDerWRTxy = numerator/denominator 
    hessian = [[doubDerWRTx, doubleDerWRTxy], [doubleDerWRTxy,doubDerWRTy]]
    return hessian


In [7]:
def runNewtonRaphson(M, N, sigmaSquared, m, n, A, T, dx, dy, delx, dely):
    min = float('inf')
    xRange = np.arange(0, M-m+1, delx)
    yRange = np.arange(0, N-n+1, dely)
    for i in xRange:
        for j in yRange:
            x_0 = i
            y_0 = j
            for z in range(10):
                hessianMatrix = calculateHessian(M, N, sigmaSquared, dx, dy, m, n, A, T, x_0, y_0)
                Gx, Gy = gradient(M, N, sigmaSquared, dx, dy, m, n, A, T, x_0, y_0)
                #G = np.array([Gx, Gy])
                denominator = hessianMatrix[0][0]*hessianMatrix[1][1]-(hessianMatrix[0][1]**2)
    
                if (denominator == 0):
                    x_0 = -1
                    y_0 = -1
                    break
                else:
                    u1 = (hessianMatrix[1][1]*Gx-hessianMatrix[1][0]*Gy)/denominator
                    u2 = (hessianMatrix[0][0]*Gy-hessianMatrix[1][0]*Gx)/denominator
                    x_0 = x_0 - u1
                    y_0 = y_0 - u2
            newObjective = objectiveFunction(M, N, sigmaSquared, m, n, A, T, x_0, y_0)
            if newObjective < min:
                point = (x_0, y_0)
                min = newObjective
    if (M < point[0]) or (0 > point[0]) or (N < point[1]) or (0 > point[1]):
        return(-1,-1)
    else:
        return point           

In [9]:
# using grid spacing of 0.2 
def main():
    delx, dely, dx, dy, sigmaSquared, M, N, bigImage, m, n, smallImage = readTxtFile("PS2_test_1.txt")
    NR = runNewtonRaphson(M, N, sigmaSquared, m, n, bigImage, smallImage, dx, dy, delx=0.2, dely=0.2)
    print("The output for txt1 is ", NR)
    delx, dely, dx, dy, sigmaSquared, M, N, bigImage, m, n, smallImage = readTxtFile("PS2_test_2.txt")
    NR = runNewtonRaphson(M, N, sigmaSquared, m, n, bigImage, smallImage, dx, dy, delx=0.2, dely=0.2)
    print("The output for txt2 is ", NR)

if __name__ == "__main__":
    main()

The output for txt1 is  (4.50000611546777, 3.5000000000000133)
The output for txt2 is  (5.386138514914719, 0.18760102600531411)
